# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Zieba  ->  S. Zieba  |  ['S. Zieba']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
Arxiv has 49 new papers today
          1 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/1 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2308.05899


extracting tarball to tmp_2308.05899...

 done.


S. Zieba  ->  S. Zieba  |  ['S. Zieba']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']


Found 55 bibliographic references in tmp_2308.05899/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.05899-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.05899) | **Potential Atmospheric Compositions of TRAPPIST-1 c constrained by  JWST/MIRI Observations at 15 $μ$m**  |
|| A. P. Lincowski, et al. -- incl., <mark>S. Zieba</mark>, <mark>L. Kreidberg</mark> |
|*Appeared on*| *2023-08-14*|
|*Comments*| *15 pages, accepted to APJL*|
|**Abstract**| The first JWST observations of TRAPPIST-1 c showed a secondary eclipse depth of 421+/-94 ppm at 15 um, which is consistent with a bare rock surface or a thin, O2-dominated, low CO2 atmosphere (Zieba et al. 2023). Here, we further explore potential atmospheres for TRAPPIST-1 c by comparing the observed secondary eclipse depth to synthetic spectra of a broader range of plausible environments. To self-consistently incorporate the impact of photochemistry and atmospheric composition on atmospheric thermal structure and predicted eclipse depth, we use a two-column climate model coupled to a photochemical model, and simulate O2-dominated, Venus-like, and steam atmospheres. We find that a broader suite of plausible atmospheric compositions are also consistent with the data. For lower pressure atmospheres (0.1 bar), our O2-CO2 atmospheres produce eclipse depths within 1$\sigma$ of the data, consistent with the modeling results of Zieba et al. (2023). However, for higher-pressure atmospheres, our models produce different temperature-pressure profiles and are less pessimistic, with 1-10 bar O2, 100 ppm CO2 models within 2.0-2.2$\sigma$ of the measured secondary eclipse depth, and up to 0.5% CO2 within 2.9$\sigma$. Venus-like atmospheres are still unlikely. For thin O2 atmospheres of 0.1 bar with a low abundance of CO2 ($\sim$100 ppm), up to 10% water vapor can be present and still provide an eclipse depth within 1$\sigma$ of the data. We compared the TRAPPIST-1 c data to modeled steam atmospheres of $\leq$ 3 bar, which are 1.7-1.8$\sigma$ from the data and not conclusively ruled out. More data will be required to discriminate between possible atmospheres, or to more definitively support the bare rock hypothesis. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2308.05899.md
    + _build/html/tmp_2308.05899/./btemp_venus.png
    + _build/html/tmp_2308.05899/./btemp_steam.png
    + _build/html/tmp_2308.05899/./btemp_o2.png
    + _build/html/tmp_2308.05899/./ptz_mix_h2o_o2.png
    + _build/html/tmp_2308.05899/./atms4.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\um}{\upmum}$</div>



<div id="title">

# Potential Atmospheric Compositions of TRAPPIST-1 c constrained by JWST/MIRI Observations at 15$\um$ 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.05899-b31b1b.svg)](https://arxiv.org/abs/2308.05899)<mark>Appeared on: 2023-08-14</mark> -  _15 pages, accepted to APJL_

</div>
<div id="authors">

A. P. Lincowski, et al. -- incl., <mark>S. Zieba</mark>, <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** The first JWST observations of TRAPPIST-1 c showed a secondary eclipse depth of 421 $\pm94$ ppm at 15 $\um$ , ${which is consistent with a bare rock surface or a thin, \ce{O2}-dominated, low \ce{CO2} atmosphere  ([ and Zieba 2023]()) .}$ Here, we further explore potential atmospheres for TRAPPIST-1 c by comparing the observed secondary eclipse depth to synthetic spectra of a broader range of plausible environments. ${ To self-consistently incorporate the impact of photochemistry and atmospheric composition on atmospheric thermal structure and predicted eclipse depth, we use a two-column climate model coupled to a photochemical model, and simulate \ce{O2}-dominated, Venus-like, and steam atmospheres.}$ We find that a broader suite of plausible atmospheric compositions are also consistent with the data.  For lower pressure atmospheres (0.1 bar), our $\ce{O2}$ - $\ce{CO2}$ atmospheres produce eclipse depths within 1 $\sigma$ of the data, consistent with the modeling results of [ and Zieba (2023)]() . ${However, for higher-pressure atmospheres, }$ our models produce different temperature-pressure profiles and are less pessimistic, with 1--10 bar $\ce{O2}$ , 100 ppm $\ce{CO2}$ models within 2.0--2.2 $\sigma$ of the measured secondary eclipse depth, and up to 0.5 \% $\ce{CO2}$ within 2.9 $\sigma$ . ${Venus-like atmospheres are still unlikely.}$ For thin $\ce{O2}$ atmospheres of 0.1 bar with a low abundance of $\ce{CO2}$ ( $\sim$ 100 ppm), up to 10 \% water vapor can be present and still provide an eclipse depth within 1 $\sigma$ of the data. ${We compared the TRAPPIST-1 c data to modeled steam atmospheres of $\leq$ 3 bar, which are 1.7--1.8$\sigma$ from the data and not conclusively ruled out. More data will be required to discriminate between possible atmospheres, or to more definitively support the bare rock hypothesis.}$

</div>

<div id="div_fig1">

<img src="tmp_2308.05899/./btemp_venus.png" alt="Fig2.1" width="33%"/><img src="tmp_2308.05899/./btemp_steam.png" alt="Fig2.2" width="33%"/><img src="tmp_2308.05899/./btemp_o2.png" alt="Fig2.3" width="33%"/>

**Figure 2. -** Brightness temperature spectra for the dayside hemisphere of all modeled environments, with points corresponding to the model spectra convolved to the F1500W filter band over the band's wavelength extent (horizontal error bars show the FWHM of the filter band). We also plot lines for 340 K (blackbody, no atmosphere, full heat redistribution), and 430 K (blackbody, no atmosphere, no heat redistribution), along with the data point measured by [ and Zieba (2023)](), which has a brightness temperature equivalent to $380\pm31$ K. _Top:_ Venus-like modeled environments. _{Middle_:} steam environments. _Bottom:_\ce{O2}-dominated environments. The Venuses are between 2.6--3.1$\sigma$ from the measured eclipse depth and the steam atmospheres are within {1.7--1.8$\sigma$} of the measured eclipse depth. The clear-sky Venuses all exhibit similar \ce{CO2} bands between 0.1--10 bar. The steam environments spanning 0.1--3 bar are nearly identical across the MIR spectrum. The \ce{O2}-dominated environments, with varying amounts of \ce{CO2}, exhibit the largest range in their spectra, and those of $\geq1$ bar also exhibit strong ozone features. For reference, the F1500W brightness temperature values from [ and Zieba (2023)]() for their best-fit desiccated \ce{O2}-\ce{CO2} atmosphere is 395 K, and their best-fit bare rock surface is 420 K. (*fig:eclipse_venus*)

</div>
<div id="div_fig2">

<img src="tmp_2308.05899/./ptz_mix_h2o_o2.png" alt="Fig5" width="100%"/>

**Figure 5. -** Global mixing ratio profiles for modeled steam (1 bar) and \ce{O2} atmospheres. \ce{H2O} and \ce{CO2} maintain evenly mixed profiles for each case. Ozone and CO are photochemically generated. As discussed in the main text, some of the cases do have CO outgassing, as seen here in the profiles.  (*fig:mixes*)

</div>
<div id="div_fig3">

<img src="tmp_2308.05899/./atms4.png" alt="Fig1" width="100%"/>

**Figure 1. -** Day-side hemisphere temperature structures for all modeled atmospheres: Venus-like (left panel), steam atmospheres (middle panel) and \ce{O2}-\ce{CO2} atmospheres (right panel). {For our modeled atmospheres, we have used a thicker line to show the layers over which the 15 $\um$  band reached optical depth of 1 (indicating the effective emission layer), for each model atmosphere. The thick dots indicate the lowest layer probed in the 15 $\um$  band.} In the right panel, we also included two example temperature-pressure profiles from [ and Zieba (2023)]() for 0.1 and 1 bar \ce{O2}-dominated atmospheres, each with 100 ppm \ce{CO2}. A wide variety of temperature profiles are possible for TRAPPIST-1 c under different atmospheric compositions. Although surface temperatures differ widely, note the similarities in {emission temperatures/pressures probed} for similar atmospheric compositions.  (*fig:atms*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.05899"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

247  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
